In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!curl https://sdk.cloud.google.com | bash

Streaming output truncated to the last 5000 lines.
google-cloud-sdk/lib/googlecloudsdk/third_party/apis/cloudresourcemanager_v2beta1.json
google-cloud-sdk/lib/googlecloudsdk/third_party/apis/cloudresourcesearch/
google-cloud-sdk/lib/googlecloudsdk/third_party/apis/cloudresourcesearch/__init__.py
google-cloud-sdk/lib/googlecloudsdk/third_party/apis/cloudresourcesearch/v1/
google-cloud-sdk/lib/googlecloudsdk/third_party/apis/cloudresourcesearch/v1/__init__.py
google-cloud-sdk/lib/googlecloudsdk/third_party/apis/cloudresourcesearch/v1/cloudresourcesearch_v1_client.py
google-cloud-sdk/lib/googlecloudsdk/third_party/apis/cloudresourcesearch/v1/cloudresourcesearch_v1_messages.py
google-cloud-sdk/lib/googlecloudsdk/third_party/apis/cloudresourcesearch_v1.json
google-cloud-sdk/lib/googlecloudsdk/third_party/apis/cloudscheduler/
google-cloud-sdk/lib/googlecloudsdk/third_party/apis/cloudscheduler/__init__.py
google-cloud-sdk/lib/googlecloudsdk/third_party/apis/cloudscheduler/v1/
google-cloud-sdk

In [ ]:
!gcloud init

Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'True'
compute:
  gce_metadata_read_timeout_sec: '0'
core:
  account: maskaravivek@gmail.com

Pick configuration to use:
 [1] Re-initialize this configuration [default] with new settings 
 [2] Create a new configuration
Please enter your numeric choice:  1

Your current configuration has been set to: [default]

You can skip diagnostics next time by using the following flag:
  gcloud init --skip-diagnostics

Network diagnostic detects and fixes local network connection issues.
Reachability Check passed.
Network diagnostic passed (1/1 checks passed).

Choose the account you would like to use to perform operations for 
this configuration:
 [1] maskaravivek@gmail.com
 [2] Log in with a new account
Please enter your numeric choice:  1

You are logged in as: [maskaravivek@gmail.com].

Pick cloud project to use: 
 [1] alou

In [ ]:
!gsutil cp gs://maskaravivek-data/data_file.csv .

Copying gs://maskaravivek-data/data_file.csv...
/ [1 files][573.1 KiB/573.1 KiB]                                                
Operation completed over 1 objects/573.1 KiB.                                    


In [ ]:
!gsutil cp gs://maskaravivek-data/test.zip .
!gsutil cp gs://maskaravivek-data/train.zip .

Copying gs://maskaravivek-data/test.zip...
- [1 files][  2.5 GiB/  2.5 GiB]   49.6 MiB/s                                   
Operation completed over 1 objects/2.5 GiB.                                      
Copying gs://maskaravivek-data/train.zip...
/ [1 files][  6.4 GiB/  6.4 GiB]   53.8 MiB/s                                   
Operation completed over 1 objects/6.4 GiB.                                      


In [ ]:
!unzip test.zip
!unzip train.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/RockClimbingIndoor/v_RockClimbingIndoor_g24_c07-0190.jpg  
  inflating: train/RockClimbingIndoor/v_RockClimbingIndoor_g13_c06-0142.jpg  
  inflating: train/RockClimbingIndoor/v_RockClimbingIndoor_g22_c02-0214.jpg  
  inflating: train/RockClimbingIndoor/v_RockClimbingIndoor_g17_c05-0125.jpg  
  inflating: train/RockClimbingIndoor/v_RockClimbingIndoor_g09_c02-0041.jpg  
  inflating: train/RockClimbingIndoor/v_RockClimbingIndoor_g11_c04-0056.jpg  
  inflating: train/RockClimbingIndoor/v_RockClimbingIndoor_g24_c03-0367.jpg  
  inflating: train/RockClimbingIndoor/v_RockClimbingIndoor_g12_c01-0068.jpg  
  inflating: train/RockClimbingIndoor/v_RockClimbingIndoor_g25_c02-0016.jpg  
  inflating: train/RockClimbingIndoor/v_RockClimbingIndoor_g10_c05-0327.jpg  
  inflating: train/RockClimbingIndoor/v_RockClimbingIndoor_g19_c02-0098.jpg  
  inflating: train/RockClimbingIndoor/v_RockClimbingIndoor_g21_c01-0071.jpg  
  inflating: 

In [ ]:
!rm test.zip
!rm train.zip

In [ ]:
"""
Process an image that we can pass to our networks.
"""
from keras.preprocessing.image import img_to_array, load_img
import numpy as np

def process_image(image, target_shape):
    """Given an image, process it and return the array."""
    # Load the image.
    h, w, _ = target_shape
    image = load_img(image, target_size=(h, w))

    # Turn it into numpy, normalize and return.
    img_arr = img_to_array(image)
    x = (img_arr / 255.).astype(np.float32)

    return x

Using TensorFlow backend.


In [ ]:
"""
Class for managing our data.
"""
import csv
import numpy as np
import random
import glob
import os.path
import sys
import operator
import threading
from keras.utils import to_categorical

class threadsafe_iterator:
    def __init__(self, iterator):
        self.iterator = iterator
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        with self.lock:
            return next(self.iterator)

def threadsafe_generator(func):
    """Decorator"""
    def gen(*a, **kw):
        return threadsafe_iterator(func(*a, **kw))
    return gen

class DataSet():

    def __init__(self, seq_length=40, class_limit=None, image_shape=(224, 224, 3)):
        """Constructor.
        seq_length = (int) the number of frames to consider
        class_limit = (int) number of classes to limit the data to.
            None = no limit.
        """
        self.seq_length = seq_length
        self.class_limit = class_limit
        self.sequence_path = os.path.join('/content/gdrive/My Drive/cv_data/sequences')
        self.max_frames = 300  # max number of frames a video can have for us to use it

        # Get the data.
        self.data = self.get_data()

        # Get the classes.
        self.classes = self.get_classes()

        # Now do some minor data cleaning.
        self.data = self.clean_data()

        self.image_shape = image_shape

    @staticmethod
    def get_data():
        """Load our data from file."""
        with open(os.path.join('data_file.csv'), 'r') as fin:
            reader = csv.reader(fin)
            data = list(reader)

        return data

    def clean_data(self):
        """Limit samples to greater than the sequence length and fewer
        than N frames. Also limit it to classes we want to use."""
        data_clean = []
        for item in self.data:
            if int(item[3]) >= self.seq_length and int(item[3]) <= self.max_frames \
                    and item[1] in self.classes:
                data_clean.append(item)

        return data_clean

    def get_classes(self):
        """Extract the classes from our data. If we want to limit them,
        only return the classes we need."""
        classes = []
        for item in self.data:
            if item[1] not in classes:
                classes.append(item[1])

        # Sort them.
        classes = sorted(classes)

        # Return.
        if self.class_limit is not None:
            return classes[:self.class_limit]
        else:
            return classes

    def get_class_one_hot(self, class_str):
        """Given a class as a string, return its number in the classes
        list. This lets us encode and one-hot it for training."""
        # Encode it first.
        label_encoded = self.classes.index(class_str)

        # Now one-hot it.
        label_hot = to_categorical(label_encoded, len(self.classes))

        assert len(label_hot) == len(self.classes)

        return label_hot

    def split_train_test(self):
        """Split the data into train and test groups."""
        train = []
        test = []
        for item in self.data:
            if item[0] == 'train':
                train.append(item)
            else:
                test.append(item)
        return train, test

    def get_all_sequences_in_memory(self, train_test, data_type):
        """
        This is a mirror of our generator, but attempts to load everything into
        memory so we can train way faster.
        """
        # Get the right dataset.
        train, test = self.split_train_test()
        data = train if train_test == 'train' else test

        print("Loading %d samples into memory for %sing." % (len(data), train_test))

        X, y = [], []
        for row in data:

            if data_type == 'images':
                frames = self.get_frames_for_sample(row)
                frames = self.rescale_list(frames, self.seq_length)

                # Build the image sequence
                sequence = self.build_image_sequence(frames)

            else:
                sequence = self.get_extracted_sequence(data_type, row)

                if sequence is None:
                    print("Can't find sequence. Did you generate them?")
                    raise

            X.append(sequence)
            y.append(self.get_class_one_hot(row[1]))

        return np.array(X), np.array(y)

    @threadsafe_generator
    def frame_generator(self, batch_size, train_test, data_type):
        """Return a generator that we can use to train on. There are
        a couple different things we can return:
        data_type: 'features', 'images'
        """
        # Get the right dataset for the generator.
        train, test = self.split_train_test()
        data = train if train_test == 'train' else test

        print("Creating %s generator with %d samples." % (train_test, len(data)))

        while 1:
            X, y = [], []

            # Generate batch_size samples.
            for _ in range(batch_size):
                # Reset to be safe.
                sequence = None

                # Get a random sample.
                sample = random.choice(data)

                # Check to see if we've already saved this sequence.
                if data_type is "images":
                    # Get and resample frames.
                    frames = self.get_frames_for_sample(sample)
                    frames = self.rescale_list(frames, self.seq_length)

                    # Build the image sequence
                    sequence = self.build_image_sequence(frames)
                else:
                    # Get the sequence from disk.
                    sequence = self.get_extracted_sequence(data_type, sample)

                    if sequence is None:
                        raise ValueError("Can't find sequence. Did you generate them?")

                X.append(sequence)
                y.append(self.get_class_one_hot(sample[1]))
            yield np.array(X), np.array(y)

    def build_image_sequence(self, frames):
        """Given a set of frames (filenames), build our sequence."""
        return [process_image(x, self.image_shape) for x in frames]

    def get_extracted_sequence(self, data_type, sample):
        """Get the saved extracted features."""
        filename = sample[2]
        path = os.path.join(self.sequence_path, filename + '-' + str(self.seq_length) + \
            '-' + data_type + '.npy')
        if os.path.isfile(path):
            return np.load(path)
        else:
            return None

    def get_frames_by_filename(self, filename, data_type):
        """Given a filename for one of our samples, return the data
        the model needs to make predictions."""
        # First, find the sample row.
        sample = None
        for row in self.data:
            if row[2] == filename:
                sample = row
                break
        if sample is None:
            raise ValueError("Couldn't find sample: %s" % filename)

        if data_type == "images":
            # Get and resample frames.
            frames = self.get_frames_for_sample(sample)
            frames = self.rescale_list(frames, self.seq_length)
            # Build the image sequence
            sequence = self.build_image_sequence(frames)
        else:
            # Get the sequence from disk.
            sequence = self.get_extracted_sequence(data_type, sample)

            if sequence is None:
                raise ValueError("Can't find sequence. Did you generate them?")

        return sequence

    @staticmethod
    def get_frames_for_sample(sample):
        """Given a sample row from the data file, get all the corresponding frame
        filenames."""
        path = os.path.join(sample[0], sample[1])
        filename = sample[2]
        images = sorted(glob.glob(os.path.join(path, filename + '*jpg')))
        return images

    @staticmethod
    def get_filename_from_image(filename):
        parts = filename.split(os.path.sep)
        return parts[-1].replace('.jpg', '')

    @staticmethod
    def rescale_list(input_list, size):
        """Given a list and a size, return a rescaled/samples list. For example,
        if we want a list of size 5 and we have a list of size 25, return a new
        list of size five which is every 5th element of the origina list."""
        assert len(input_list) >= size

        # Get the number to skip between iterations.
        skip = len(input_list) // size

        # Build our new output.
        output = [input_list[i] for i in range(0, len(input_list), skip)]

        # Cut off the last one if needed.
        return output[:size]

    def print_class_from_prediction(self, predictions, nb_to_return=5):
        """Given a prediction, print the top classes."""
        # Get the prediction for each label.
        label_predictions = {}
        for i, label in enumerate(self.classes):
            label_predictions[label] = predictions[i]

        # Now sort them.
        sorted_lps = sorted(
            label_predictions.items(),
            key=operator.itemgetter(1),
            reverse=True
        )

        # And return the top N.
        for i, class_prediction in enumerate(sorted_lps):
            if i > nb_to_return - 1 or class_prediction[1] == 0.0:
                break
            print("%s: %.2f" % (class_prediction[0], class_prediction[1]))

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Dropout, ZeroPadding3D, LSTM, Conv2D, MaxPooling3D, Input, Activation, TimeDistributed

def add_default_block(model, kernel_filters, init, reg_lambda):
    # conv
    model.add(TimeDistributed(Conv2D(kernel_filters, (3, 3), padding='same')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(Activation('relu')))
    # conv
    model.add(TimeDistributed(Conv2D(kernel_filters, (3, 3), padding='same')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(Activation('relu')))
    # max pool
    model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

    return model

In [ ]:
"""
A collection of models we'll use to attempt to classify videos.
"""
from tensorflow.keras.layers import Dense, Flatten, Dropout, LSTM, Conv2D, MaxPooling2D, Input, Activation, TimeDistributed, BatchNormalization, Input
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam
from keras.regularizers import l2
from keras.models import Model


from collections import deque
import sys

seq_length = 16
features_length=2048
nb_classes= 101
batch_size=8

metrics = ['accuracy']
if nb_classes >= 10:
    metrics.append('top_k_categorical_accuracy')

initialiser = 'glorot_uniform'
input_shape = (seq_length, 240, 320, 3)
reg_lambda  = 0.001

model = Sequential()

input_batch = Input(shape = input_shape, batch_size = batch_size)

# first (non-default) block
model.add(TimeDistributed(Conv2D(32, (7, 7), strides=(2, 2), padding='same'), input_shape=input_shape))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Conv2D(32, (3,3))))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

# 2nd-5th (default) blocks
model = add_default_block(model, 64,  init=initialiser, reg_lambda=reg_lambda)
model = add_default_block(model, 128, init=initialiser, reg_lambda=reg_lambda)
model = add_default_block(model, 256, init=initialiser, reg_lambda=reg_lambda)
model = add_default_block(model, 512, init=initialiser, reg_lambda=reg_lambda)

# LSTM output head
model.add(TimeDistributed(Flatten()))
model.add(LSTM(256, return_sequences=False, dropout=0.5))
model.add(Dense(nb_classes, activation='softmax'))

optimizer = Adam(lr=1e-5, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=optimizer,
                    metrics=metrics)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 16, 120, 160, 32)  4736      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 16, 120, 160, 32)  128       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 16, 120, 160, 32)  0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 16, 118, 158, 32)  9248      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 16, 118, 158, 32)  128       
_________________________________________________________________
time_distributed_5 (TimeDist (None, 16, 118, 158, 32)  0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 16, 59, 79, 32)   

In [ ]:
"""
Train our RNN on extracted features or images.
"""
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger
import time
import os.path

class_limit = None
nb_epoch=100
image_shape = (240, 320, 3)
# Helper: Save the model.
checkpointer = ModelCheckpoint('lrcn-images.{epoch:03d}-{val_loss:.3f}.hdf5', verbose=1, save_best_only=True)

# Helper: TensorBoard
tb = TensorBoard(log_dir=os.path.join('data', 'logs', 'lstm'))

# Helper: Stop when we stop learning.
early_stopper = EarlyStopping(patience=5)

# Helper: Save results.
timestamp = time.time()
csv_logger = CSVLogger(os.path.join('data', 'logs', 'lstm-training-' + \
    str(timestamp) + '.log'))


# Get the data and process it.
data = DataSet(
            seq_length=seq_length,
            class_limit=class_limit,
            image_shape=image_shape
        )

# Get samples per epoch.
# Multiply by 0.7 to attempt to guess how much of data.data is the train set.
steps_per_epoch = (len(data.data) * 0.7) // batch_size

generator = data.frame_generator(batch_size, 'train', 'images')
val_generator = data.frame_generator(batch_size, 'test', 'images')

print(generator.__iter__)

<bound method threadsafe_iterator.__iter__ of <__main__.threadsafe_iterator object at 0x7f3cb01bb240>>


In [ ]:
model.fit_generator(
          generator=generator,
          steps_per_epoch=steps_per_epoch,
          epochs=nb_epoch,
          verbose=1,
          callbacks=[tb, early_stopper, csv_logger, checkpointer],
          validation_data=val_generator,
          validation_steps=40,
          workers=4)

Instructions for updating:
Please use Model.fit, which supports generators.
Creating train generator with 8599 samples.
Epoch 1/100
1052/1052 [==============================] - ETA: 0s - loss: 4.4090 - accuracy: 0.0631 - top_k_categorical_accuracy: 0.1774Creating test generator with 3424 samples.

Epoch 00001: val_loss improved from inf to 4.29583, saving model to lrcn-images.001-4.296.hdf5
1052/1052 [==============================] - 552s 525ms/step - loss: 4.4090 - accuracy: 0.0631 - top_k_categorical_accuracy: 0.1774 - val_loss: 4.2958 - val_accuracy: 0.0594 - val_top_k_categorical_accuracy: 0.2250
Epoch 2/100
1052/1052 [==============================] - ETA: 0s - loss: 3.8056 - accuracy: 0.1776 - top_k_categorical_accuracy: 0.3967
Epoch 00002: val_loss improved from 4.29583 to 3.98903, saving model to lrcn-images.002-3.989.hdf5
1052/1052 [==============================] - 529s 503ms/step - loss: 3.8056 - accuracy: 0.1776 - top_k_categorical_accuracy: 0.3967 - val_loss: 3.9890 - val